In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(
    temperature=0.2,
    model_name="gpt-3.5-turbo-0125",
    openai_api_key=openai_api_key,
    max_tokens=100,
)

In [2]:
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

from langchain_community.document_loaders import PyPDFLoader
loader= PyPDFLoader("E:\Habibi\DB Chatbot\policies\MZbankpolicies.pdf")
document=loader.load()
print(document)


[Document(page_content="MEEZAN BANK POLICIES & PROCEDURES FOR NEW \nSTAFF  \nOnboarding Procedures  \nObjective:  To ensure a smooth integration of new employees into the bank, \nfacilitating understanding of the bank's culture, processes, and expectations.  \n1. Pre-Onboarding  Activities:  \n• Offer  Letter  and Documentation:  Send out offer letters and necessary \nemployment forms including bank details, tax forms, and identity \nverification documents.  \n• Background  Checks:  Conduct comprehensive background checks \nfollowing local legal guidelines.  \n2. Orientation  Program:  \n• First Day Agenda:  Welcome session, introduction to team and key \ndepartments, issuance of employee ID, and workstation setup.  \n• Training  Sessions:  Schedule initial training sessions covering bank \noperations, compliance standards, and Islamic banking principles.  \n• Mentor  Assignment:  Assign a mentor for at least the first three \nmonths to guide the new employee through their initial peri

In [3]:
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(document)
embeddings = OpenAIEmbeddings()

In [4]:
# pip install chromadb==0.4.24

vectorstore = Chroma.from_documents(documents=texts,embedding=embeddings)

In [8]:
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001B76E00C970>)

In [10]:
prompt = ChatPromptTemplate.from_template(
"""Answer the question based only on the following context:
{context}

Question: {question}
"""    
)

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | StrOutputParser()

chain.invoke("Tell me about this document?")

'Based on the context provided, the document appears to be a collection of various policies implemented by a company, specifically MZbank. These policies cover a range of topics such as remote work, anti-discrimination and harassment, health and safety, change management, conflict resolution, data protection, sustainability, volunteerism and community engagement, setting goals, conducting performance reviews, and parental leave benefits. Each policy outlines the objectives, procedures, guidelines, and initiatives related to the specific topic it addresses.'

In [11]:
from langchain_core.runnables import RunnableLambda

def add_one(x: int) -> int:
    return x + 1

def mul_two(x: int) -> int:
    return x * 2

def mul_three(x: int) -> int:
    return x * 3

runnable_1 = RunnableLambda(add_one)
runnable_2 = RunnableLambda(mul_two)
runnable_3 = RunnableLambda(mul_three)

sequence = runnable_1 | {  # this dict is coerced to a RunnableParallel
    "mul_two": runnable_2,
    "mul_three": runnable_3,
}
# Or equivalently:
# sequence = runnable_1 | RunnableParallel(
#     {"mul_two": runnable_2, "mul_three": runnable_3}
# )
# Also equivalently:
# sequence = runnable_1 | RunnableParallel(
#     mul_two=runnable_2,
#     mul_three=runnable_3,
# )

sequence.invoke(1)
await sequence.ainvoke(1)

sequence.batch([1, 2, 3])
await sequence.abatch([1, 2, 3])

[{'mul_two': 4, 'mul_three': 6},
 {'mul_two': 6, 'mul_three': 9},
 {'mul_two': 8, 'mul_three': 12}]

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
joke_chain = (
    ChatPromptTemplate.from_template("tell me a joke about {topic}")
    | model
)
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}")
    | model
)

runnable = RunnableParallel(joke=joke_chain, poem=poem_chain)

# Display stream
output = {key: "" for key, _ in runnable.output_schema()}
for chunk in runnable.stream({"topic": "bear"}):
    for key in chunk:
        output[key] = output[key] + chunk[key].content
    print(output)  # noqa: T201

{'joke': '', 'poem': ''}
{'joke': 'Why', 'poem': ''}
{'joke': 'Why did', 'poem': ''}
{'joke': 'Why did', 'poem': ''}
{'joke': 'Why did the', 'poem': ''}
{'joke': 'Why did the bear', 'poem': ''}
{'joke': 'Why did the bear', 'poem': 'In'}
{'joke': 'Why did the bear break', 'poem': 'In'}
{'joke': 'Why did the bear break up', 'poem': 'In'}
{'joke': 'Why did the bear break up with', 'poem': 'In'}
{'joke': 'Why did the bear break up with his', 'poem': 'In'}
{'joke': 'Why did the bear break up with his', 'poem': 'In the'}
{'joke': 'Why did the bear break up with his', 'poem': 'In the forest'}
{'joke': 'Why did the bear break up with his girlfriend', 'poem': 'In the forest'}
{'joke': 'Why did the bear break up with his girlfriend?', 'poem': 'In the forest'}
{'joke': 'Why did the bear break up with his girlfriend?', 'poem': 'In the forest deep'}
{'joke': 'Why did the bear break up with his girlfriend?', 'poem': 'In the forest deep and'}
{'joke': 'Why did the bear break up with his girlfriend?',

In [14]:
from langchain_community.llms.fake import FakeStreamingListLLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import SystemMessagePromptTemplate
from langchain_core.runnables import Runnable
from operator import itemgetter

prompt = (
    SystemMessagePromptTemplate.from_template("You are a nice assistant.")
    + "{question}"
)
llm = FakeStreamingListLLM(responses=["foo-lish"])

chain: Runnable = prompt | llm | {"str": StrOutputParser()}

chain_with_assign = chain.assign(hello=itemgetter("str") | llm)

print(chain_with_assign.input_schema.schema())
# {'title': 'PromptInput', 'type': 'object', 'properties':
{'question': {'title': 'Question', 'type': 'string'}}
print(chain_with_assign.output_schema.schema()) #
{'title': 'RunnableSequenceOutput', 'type': 'object', 'properties':
{'str': {'title': 'Str',
'type': 'string'}, 'hello': {'title': 'Hello', 'type': 'string'}}}

{'title': 'PromptInput', 'type': 'object', 'properties': {'question': {'title': 'Question', 'type': 'string'}}}
{'title': 'RunnableSequenceOutput', 'type': 'object', 'properties': {'str': {'title': 'Str', 'type': 'string'}, 'hello': {'title': 'Hello', 'type': 'string'}}}


{'title': 'RunnableSequenceOutput',
 'type': 'object',
 'properties': {'str': {'title': 'Str', 'type': 'string'},
  'hello': {'title': 'Hello', 'type': 'string'}}}

In [16]:
from langchain_core.runnables import RunnableLambda

def add_one(x: int) -> int:
    return x + 1

def mul_two(x: int) -> int:
    return x * 2

runnable_1 = RunnableLambda(add_one)
runnable_2 = RunnableLambda(mul_two)
sequence = runnable_1.pipe(runnable_2)
# Or equivalently:
# sequence = runnable_1 | runnable_2
# sequence = RunnableSequence(first=runnable_1, last=runnable_2)
sequence.invoke(1)
await sequence.ainvoke(1)
# -> 4

sequence.batch([1, 2, 3])
await sequence.abatch([1, 2, 3])

[4, 6, 8]

In [ ]:
# (1+1)*2 = 4
# (2+1)*2 = 6
# (3+1)*2 = 8

In [21]:
from typing import Iterator

from langchain_core.runnables import RunnableGenerator


def _generate_immediate_error(input: Iterator) -> Iterator[str]:
    raise ValueError()
    yield ""


def _generate(input: Iterator) -> Iterator[str]:
    yield from "foo bar"

runnable = RunnableGenerator(_generate_immediate_error)

runnable = RunnableGenerator(_generate_immediate_error).with_fallbacks(
    [RunnableGenerator(_generate)]
    )
print(''.join(runnable.stream({}))) #foo bar

foo bar
